In [ ]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
url='https://www.daraz.pk/products/12-8gb-128gb-5000mah-33-i421379488-s1989920618.html?spm=a2a0e.searchlist.sku.1.29313de2gohjjQ&search=1'

In [ ]:
import time
import pandas as pd

driver.get(url)

temp = dict()
temp['Name'] = []
temp['Review text'] = []
temp['sentiment']=[]

#scrolling down the page to load reviews
driver.execute_script("window.scrollTo(0, 500);")
time.sleep(1)
driver.execute_script("window.scrollTo(500, 800);")
time.sleep(1)
driver.execute_script("window.scrollTo(800, 1000);")
time.sleep(2)
driver.execute_script("window.scrollTo(1000, 1300);")
time.sleep(2)
driver.execute_script("window.scrollTo(1300, 1600);")
time.sleep(1)

#looping through review pages
for page in range(0,3):
    try:
        # Finding the container for reviews
        reviews_container = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[1]')
    except NoSuchElementException:
        print('No Reviews')
        break
        #finding all individual reviews on the current page
    reviews = reviews_container.find_elements(By.CLASS_NAME,'item')

    #looping through each review on the current page
    for i in range(len(reviews)):
        #checking if the review text is not empty
        if reviews[i].find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({i+1}) > div.item-content > div.content").text != '':
            #extracting and appending reviewer name and review text
            temp['Name'].append(reviews[i].find_element(By.CLASS_NAME,'middle').find_element(By.TAG_NAME,'span').text[3:])
            temp['Review text'].append(reviews[i].find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({i+1}) > div.item-content > div.content").text)
            temp['sentiment'].append('x')
    try:
        #clicking the next button if it exists and not on the third page
        if(page != 2):
            nxt_button = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/button[2]')
            driver.execute_script("arguments[0].click();", nxt_button)
            time.sleep(5)
    except NoSuchElementException:
        print('No More Pages')
        break
#converting to dataframe
reviews = pd.DataFrame(temp)
reviews.to_csv('Q4_data.csv')
reviews

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

df=pd.read_csv('Q4.csv')
#initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

#loop through each row and update sentiments column with actual sentiment scores
for index, row in df.iterrows():
    sentiment_score = sia.polarity_scores(row['reviews'])['compound']
    df.at[index, 'sentiments'] = sentiment_score

df